In [1]:
using Revise
using KLIEPInference
using Distributions
using LinearAlgebra

In [ ]:
θ = zeros(5, 5)
spl = IsingSampler(θ)
X = rand(spl, 100)
Y = rand(spl, 500)
Ψx = Ψising(X)
Ψy = Ψising(Y);

In [ ]:
#θhat = KLIEP(Ψx, Ψy,  Mosek_KLIEP())
@time θhat = KLIEP(Ψx, Ψy,  SCS_KLIEP())

In [ ]:
@time θhat1 = KLIEP(Ψx, Ψy,  CD_KLIEP())
norm(θhat1 - θhat, Inf)

In [2]:
θx = zeros(5, 5)
splx = IsingSampler(θx)
X = rand(splx, 500)

θy = zeros(5, 5)
θy[1, 2] = .3
θy[2, 1] = .3
θy[3, 4] = .5
θy[4, 3] = .5
sply = IsingSampler(θy)
Y = rand(sply, 1000)

Ψx = Ψising(X)
Ψy = Ψising(Y);

In [ ]:
θhat = KLIEP(Ψx, Ψy,  SCS_KLIEP())

In [3]:
@time θhat = spKLIEP(Ψx, Ψy,  0.1, SCS_KLIEP())

 10.599376 seconds (28.32 M allocations: 1.407 GiB, 5.99% gc time)
----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 13040, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 1021, constraints m = 3021
Cones:	linear vars: 21
	exp vars: 3000, dual exp vars: 0
Setup time: 4.92e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 9.03e-01  9.88e-01  4.10e-01 -3.98e+01 -1.64e+01  1.54e-14  1.22e-02 
   100| 5.71e-03  1.96e-02  4.02e-02 -3.49e-01 -2.83e-01  8.21e-15  7

10-element Array{Float64,1}:
 -0.20969804136138612  
  2.7111475020620737e-5
 -3.212943501492139e-6 
 -1.446906768886503e-5 
 -8.348191908894331e-6 
 -2.3037773307421434e-5
  1.586796965141589e-6 
 -0.3380769156241899   
  9.813813077445105e-6 
 -2.038708221569357e-5 

In [4]:
@time θhat1 = spKLIEP(Ψx, Ψy,  0.1, CD_KLIEP())

  0.726354 seconds (2.23 M allocations: 111.600 MiB, 7.08% gc time)


10-element Array{Float64,1}:
 -0.20971291604296488
  0.0                
  0.0                
  0.0                
  0.0                
  0.0                
  0.0                
 -0.33808155284420544
  0.0                
  0.0                

In [5]:
norm(θhat1 - θhat, Inf)

2.7111475020620737e-5

In [11]:
C = KLIEP_Hessian(θhat1, Ψy)

10×10 Array{Float64,2}:
  0.995904     0.0331385   -0.0187938   …  -0.0262209    0.0199699  
  0.0331385    0.997156     0.112018       -0.0423646   -0.0103044  
 -0.0187938    0.112018     1.0            -0.00941723  -0.0419767  
 -0.00765903  -0.00940658  -0.0165762      -0.0536294   -0.000360724
 -0.0552309    0.0655852    0.0168627      -0.0101328   -0.0165832  
  0.00153721   0.0158716    0.0640063   …  -0.0172109   -0.0106573  
 -0.0413202   -0.0272314    0.0189115       0.0296513   -0.018944   
  0.00970471   0.00630857  -0.0533661      -0.0157876   -0.00530961 
 -0.0262209   -0.0423646   -0.00941723      0.999949     0.111881   
  0.0199699   -0.0103044   -0.0419767       0.111881     0.999725   

In [13]:
Hinv_row(C, 1, 0.1)

10-element Array{Float64,1}:
 0.9125715358592461
 0.0               
 0.0               
 0.0               
 0.0               
 0.0               
 0.0               
 0.0               
 0.0               
 0.0               

In [14]:
inv(C)

10×10 Array{Float64,2}:
  1.01194     -0.0376668     0.0212071   …   0.02717     -0.0211485 
 -0.0376668    1.02398      -0.114938        0.0398342    0.00134716
  0.0212071   -0.114938      1.02302         0.00229116   0.040299  
  0.00836781   0.00615051    0.0191261       0.0575304   -0.00636738
  0.0592331   -0.0668934     0.00118765      0.00743421   0.0133233 
 -0.00805224   0.000128811  -0.0659413   …   0.0126065    0.00506552
  0.0388285    0.0262975    -0.0222994      -0.0334074    0.0217264 
 -0.00598666  -0.0127874     0.0579512       0.0147893    0.0064648 
  0.02717      0.0398342     0.00229116      1.01988     -0.114449  
 -0.0211485    0.00134716    0.040299       -0.114449     1.01593   